<a href="https://colab.research.google.com/github/MadsNL/AMLMain/blob/Laura/AML_IceCores_combined_types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! gdown --id 1paSsYReykEVDPX8pA2SUf7Gr7Lptq-PX
! unzip Insoluables_TrainingData26May2022.zip
!git clone https://github.com/MadsNL/AMLpackages.git
import sys
sys.path.append('/content/AMLpackages/')
from load_data import training_set, not_training_set
!pip install pyyaml h5py 
!pip install umap-learn

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1paSsYReykEVDPX8pA2SUf7Gr7Lptq-PX
To: /content/Insoluables_TrainingData26May2022.zip
100% 726M/726M [00:03<00:00, 202MB/s]
Archive:  Insoluables_TrainingData26May2022.zip
replace Insoluables_TrainingData/train/camp.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import umap
import seaborn as sb
import lightgbm as lgb

In [ ]:
sup_path = '/content/Insoluables_TrainingData/train/'
sup_im_path = '/content/Insoluables_TrainingData/train/'
sup_df = pd.read_csv(sup_path+'train.csv')
sup = training_set(sup_df, sup_im_path)

In [ ]:
# save image_size
im_e, _, _ = sup.__getitem__(0)
image_size = im_e.shape[0]
image_size

In [ ]:
# create encoder training
nr_tot_images = len(sup.labels)
x_train = []
x_train_lab = []
x_train_feats = []
nr_ims = 10000 # 10-15k images are the max we can load with the memory we have
rand_idx = np.arange(0,nr_tot_images)
rand_idx = np.random.choice(rand_idx, replace=False, size=nr_ims)
for i in range(nr_ims):
    im, lab, feat = sup.__getitem__(rand_idx[i])
    x_train.append(im)
    x_train_lab.append(lab)
    x_train_feats.append(feat)
x_train = np.array(x_train)
x_train_lab = np.array(x_train_lab)
x_train_feats = np.array(x_train_feats)

In [ ]:
x_train.shape

In [ ]:
x_train_lab_4classes = x_train_lab.copy()


In [ ]:
x_train_lab_4classes[x_train_lab_4classes==3] = 0  # ash
x_train_lab_4classes[x_train_lab_4classes==4] = 1
x_train_lab_4classes[x_train_lab_4classes==5] = 3 # what is qsup - pollen? 
x_train_lab_4classes[x_train_lab_4classes==6] = 0

len_labels = 4

In [ ]:
latent_dim = 64
image_size= 128
unit_n = 32

encoder_input = keras.Input(shape=(image_size,image_size,1))
x = keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2,2), activation='relu')(encoder_input)
x = keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2,2), activation='relu')(x)
x = keras.layers.Flatten()(x)
encoder_output = keras.layers.Dense(latent_dim)(x)

encoder = keras.Model(encoder_input, encoder_output, name='encoder')

decoder_input = keras.layers.InputLayer(input_shape=latent_dim)(encoder_output)
x = keras.layers.Dense(units=unit_n*unit_n*32, activation=tf.nn.relu)(decoder_input)
x = keras.layers.Reshape(target_shape=(unit_n, unit_n, 32))(x)
x = keras.layers.Conv2DTranspose(
    filters=64, kernel_size=3, strides=2, padding='same',
    activation='relu')(x)
x = keras.layers.Conv2DTranspose(
    filters=32, kernel_size=3, strides=2, padding='same',
    activation='relu')(x)
# No activation
decoder_output = keras.layers.Conv2DTranspose(
    filters=1, kernel_size=3, strides=1, padding='same')(x)

autoencoder = keras.Model(encoder_input, decoder_output, name='autoencoder')
autoencoder.summary()

opt = keras.optimizers.Adam(lr=0.001, decay=1e-6)

encoder.compile(opt, loss='mse')
autoencoder.compile(opt, loss='mse')

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)
autoencoder.fit(x_train, x_train, epochs=200, batch_size=32, validation_split=0.1, callbacks=[callback])

In [ ]:
input_length = len(x_train_feats[0]) + latent_dim
dim_last_hidden = 128

dense_input = keras.Input(shape=(input_length))
x = keras.layers.Dense(128, activation='relu')(dense_input)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.BatchNormalization(momentum=0.9, center=True, scale=True)(x)
x = keras.layers.Dense(128*2, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.BatchNormalization(momentum=0.9, center=True, scale=True)(x)
last_hidden = keras.layers.Dense(dim_last_hidden, activation='relu')(x)


dense2_input = keras.layers.InputLayer(input_shape=dim_last_hidden)(last_hidden)
x = keras.layers.Dropout(0.2)(dense2_input)
x = keras.layers.BatchNormalization(momentum=0.9, center=True, scale=True)(x)
dense_output = keras.layers.Dense(7, activation='softmax')(x)

last_hidden_model = keras.Model(dense_input, last_hidden, name='mellem_model')
dense_model = keras.Model(dense_input, dense_output, name='dense_model')

dense_model.summary()

opt = keras.optimizers.Adam(lr=0.001, decay=1e-6)

last_hidden_model.compile(opt, loss='SparseCategoricalCrossentropy', metrics=['accuracy'])
dense_model.compile(opt, loss='SparseCategoricalCrossentropy', metrics=['accuracy'])


In [ ]:
n_epochs = 200
batch_size = 20
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 3, min_delta = 0.0001)

encode_scalars = np.concatenate((encoder.predict(x_train),x_train_feats), axis=1)

history = dense_model.fit(encode_scalars,x_train_lab,validation_split=0.2,batch_size=batch_size, epochs= n_epochs, callbacks=[callback])

INFO:tensorflow:Assets written to: encoder/assets


In [ ]:
y_last_hid = last_hidden_model.predict(encode_scalars)

map = umap.UMAP(n_components=2) #x and y axis
y = map.fit_transform(y_last_hid)

plt.figure()
sb.scatterplot(x=y[:, 0], y=y[:, 1], hue = x_train_lab, palette = 'tab10')
plt.show()